In [4]:
import pandas as pd
import os

# np.load("/content/drive/My Drive/ModelingAllLogs (1)/template2vec_hdfs_logs.npy",allow_pickle=True)
corpus_fake = pd.read_csv("../dataset/Fake.csv")
corpus_fake = corpus_fake[[True if len(x) > 60 else False for x in corpus_fake["text"].values]]
corpus_true = pd.read_csv("../dataset/True.csv")
corpus_true = corpus_true[[True if len(x) > 60 else False for x in corpus_true["text"].values]]


In [5]:
month_to_number = {"December":12,"November":11,"October":10,"September":9,"August":8,"July":7,"June":6,
                   "May":5,"April":3,"March":3,"February":2,"January":1,"Feb":2,"Dec":12,"Nov":11,"Oct":10,
                   "Sep":9,"Aug":8,"Jul":7,"Jun":6,"Apr":4,"Mar":3,"Jan":1}

dates = []
to_delete= []
for i in range(corpus_fake.shape[0]):
   t = corpus_fake.date.iloc[i]
   try:
      if "-" in t:
        p = pd.to_datetime(t.split("-")[0] +":" + str(month_to_number[t.split("-")[1]]) + ":"+ t.split("-")[-1],format="%d:%m:%y")
      else:     
        p =pd.to_datetime(t.split(" ")[1][:-1] +":" + str(month_to_number[t.split(" ")[0]]) + t.split(",")[-1],format='%d:%m %Y')
      dates.append(p)
   except:
     to_delete.append(i)
for i in to_delete:
  corpus_fake.drop(index = i,inplace=True)
corpus_fake["date"] = dates

dates = []
to_delete= []
for i in range(corpus_true.shape[0]):
    t = corpus_true.date.iloc[i]
    try:
        if "-" in t:
            p = pd.to_datetime(t.split("-")[0] +":" + str(month_to_number[t.split("-")[1]]) + ":"+ t.split("-")[-1],format="%d:%m:%y")
        else:     
            p =pd.to_datetime(t.split(" ")[1][:-1] +":" + str(month_to_number[t.split(" ")[0]]) + t.split(",")[-1][:-1],format='%d:%m %Y')
        dates.append(p)
    except:
        print(i)
        to_delete.append(i)
for i in to_delete:
    corpus_true.drop(index = i,inplace=True)
corpus_true["date"] = dates


In [6]:
fake_news_us_el = corpus_fake[(corpus_fake["date"] > pd.to_datetime('01:08:2016', format='%d:%m:%Y')) & 
            (corpus_fake["date"] < pd.to_datetime('12:12:2016', format='%d:%m:%Y')) &
            (corpus_fake["subject"] == "politics")]

true_news_us_el = corpus_true[(corpus_true["date"] > pd.to_datetime('22:09:2016', format='%d:%m:%Y')) & 
            (corpus_true["date"] < pd.to_datetime('27:11:2016', format='%d:%m:%Y')) &
            (corpus_true["subject"] == "politicsNews")]

In [8]:
print(true_news_us_el.shape)
fake_news_us_el.shape


(1005, 4)


(1002, 4)

In [224]:
fake_news_us_el["label"] = [1] * fake_news_us_el.shape[0]
true_news_us_el["label"] = [0] * true_news_us_el.shape[0]
# all_data = fake_news_us_el.append(true_news_us_el)
# all_data

/home/laze/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/laze/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [5]:
print("Total true news "+str(fake_news_us_el.shape[0]))
print("Total fake news "+str(true_news_us_el.shape[0]))


Total true news 1002
Total fake news 1005


In [23]:
from nltk import ngrams
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
lemmetizer = WordNetLemmatizer()
tokenizer = RegexpTokenizer(r"\w+")

text_prepared_fake = []
for t in fake_news_us_el.text.values.tolist():
    v= []
    for w in tokenizer.tokenize(t):
        w_lem = lemmetizer.lemmatize(w.lower())
        if not w_lem.isdigit() and w_lem not in stopwords.words('english') and not bool(re.search(r'\d', w_lem)):
            v.append(w_lem)
    text_prepared_fake.append(v)
    
text_prepared_true = []
for t in true_news_us_el.text.values.tolist():
    v= []
    for w in tokenizer.tokenize(t):
        w_lem = lemmetizer.lemmatize(w.lower())
        if not w_lem.isdigit() and w_lem not in stopwords.words('english') and not bool(re.search(r'\d', w_lem)):
            v.append(w_lem)
    text_prepared_true.append(v)

In [7]:
from nltk import ngrams
bigrams = [list(ngrams(t,2)) for t in text_prepared_fake[:]]
all_bigrams = set()

for t in bigrams:
    for b_gram in t:
        all_bigrams.add(" ".join(b_gram))

ImportError: No module named nltk

In [261]:
top_1000_bigrams = list(get_most_frequent_terms(bigrams,5000).keys())
top_2000_unigrams = list(get_most_frequent_terms(text_prepared[2:],2000).keys())

In [238]:
len(bigrams)

1002

In [263]:
tf.__version__

'2.0.0-rc1'

In [262]:
vectors,not_found = compute_td_idf_fatures(top_1000_bigrams,bigrams)

<h1>LSTM</h1>

<h1>VAE</h1>

In [ ]:
model = create_and_train_vae_model(np.asarray(vectors))

In [259]:
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
from tensorflow.keras import backend as K



def create_and_train_vae_model(train_data,epochs = 5):
    original_dimension=train_data.shape[1]
    # original_dimension_rec=len(include)
    original_dimension_rec=train_data.shape[1]
    train_data = train_data.reshape(train_data.shape[0],1,original_dimension)

    hidden_unit_dimension=64
    latent_dim=30
    shape=(1,original_dimension)
    batch_shape=32
    #We assume that the prior distribution of the latent space  is isotropic gausiian distribution
    prior=tfp.distributions.Independent(tfp.distributions.Normal(loc=tf.zeros(latent_dim),scale=1),
                                                     reinterpreted_batch_ndims=1)

    #encoder
    encoder=tfk.Sequential()
    encoder.add(tfkl.InputLayer(input_shape=shape))
    encoder.add(tfkl.Dense(units=hidden_unit_dimension,activation='tanh',kernel_regularizer='l2',activity_regularizer='l2'))
    encoder.add(tfkl.Dense(units=hidden_unit_dimension//2,activation='tanh',kernel_regularizer='l2'))
    encoder.add(tfkl.Dense(units=tfp.layers.MultivariateNormalTriL.params_size(latent_dim),activation='tanh',kernel_regularizer='l2'))
    encoder.add(tfp.layers.MultivariateNormalTriL(latent_dim,
                                                  activity_regularizer=tfp.layers.KLDivergenceRegularizer(prior,
                                                                                                              weight=1.0)))
    # encoder=tfk.Model(input_layer,layers)

    #decoder
    decoder=tfk.Sequential()
    decoder.add(tfkl.InputLayer(input_shape=[latent_dim]))
    decoder.add(tfkl.Reshape([1,1,latent_dim]))
    decoder.add(tfkl.Dense(units=hidden_unit_dimension,activation='tanh',kernel_regularizer='l2',activity_regularizer='l2'))
    decoder.add(tfkl.Dense(units=hidden_unit_dimension//2,activation='tanh'))
    decoder.add(tfkl.Dense(units=tfp.layers.MultivariateNormalTriL.params_size(original_dimension_rec),activation='tanh'))
    decoder.add(tfp.layers.MultivariateNormalTriL(original_dimension_rec))


    vae=tfk.Model(encoder.inputs,outputs=decoder(encoder.outputs[0]))

    def mahalanobis_distance(x,rv_x):

            curr_batch_shape=K.int_shape(x)[0]
            covariance=rv_x.covariance()
            sample_vector=tf.reduce_mean(rv_x.sample(5),axis=0)
            if(curr_batch_shape is None):
                curr_batch_shape=-1
            elif(not check_PSD(covariance.numpy())):
                    covariance=nearPSDTensorflow(covariance)

            inv_cov_matrix = tf.reshape(tf.linalg.inv(covariance),[curr_batch_shape,original_dimension_rec,original_dimension_rec])
            diff = tf.reshape(tf.math.subtract(tf.reshape(x,[curr_batch_shape,1,1,original_dimension_rec]),sample_vector),[curr_batch_shape,original_dimension_rec])
            matVec=tf.linalg.matvec(inv_cov_matrix,diff)    
            l1=tf.reduce_sum(tf.multiply(matVec,diff),axis=1)
            return l1


    def check_PSD(M):
        return np.all(np.linalg.eigvals(M)>=0)
    vae.compile(optimizer='adam',loss=mahalanobis_distance)
    vae.fit(train_data,train_data,batch_size=32,epochs = epochs)


In [236]:
np.asarray(vectors).shape

(146, 10000)

In [240]:
import numpy as np
def compute_td_idf_fatures(vocabulary :list, documents :list):
    word_to_index = {k:v for v,k in enumerate(top_1000_bigrams)}
    not_found = []
    td_idf_features = []
    i = 0
    for doc in documents:
        vector = np.zeros(len(vocabulary))
        #computing idf
        idf_per_word = {}
        j = 0
        for w in set(doc):
            count = 1
            for d in documents:
                if w in d:
                    count+=1
            idf_per_word[w] = np.log(len(documents)/count)
        #computing tf
        for w in doc:
            if w not in word_to_index:
                not_found.append(w)
                continue
            idx = word_to_index[w]
            vector[idx] = vector[idx] + 1
        vector = vector/len(doc)        
        for w in doc:
            if w not in word_to_index:
                continue
            idx = word_to_index[w]
            vector[idx] = vector[idx] * idf_per_word[w]
        td_idf_features.append(vector)
    return td_idf_features,not_found

In [11]:
def get_most_frequent_terms(terms,top_n):
    term_to_freq = {}
    for t in terms:
        for w in t:
            if w in term_to_freq:
                term_to_freq[w] = term_to_freq[w] + 1
            else:
                term_to_freq[w] = 1
     
    res = {k: v for k, v in sorted(term_to_freq.items(), key=lambda item: item[1],reverse=True)}
    return {k:term_to_freq[k] for k in list(res)[:top_n]}